In [103]:
import pydocparser
import requests
from bs4 import BeautifulSoup
import webbrowser
import re

In [104]:

api_key = 'a97eb5d696857fea8bfa9e57fb9994017a3bb7da'
parser_id = 'paper_parser'

parser = pydocparser.Parser()
parser.login(api_key)
result = parser.ping()
print(result)

pong


In [105]:
id = parser.upload_file_by_path("data_samples/electricity-theft-detection-using-machine-learning-IJERTCONV10IS04024.pdf", "paper_parser") 

In [106]:
data = parser.get_one_result("paper_parser", id)
data

[{'id': 'c6e924b6ad0cadbaecc66623c208609d',
  'document_id': '58ad73d3ccadac478dd59d2e607039f5',
  'remote_id': '',
  'file_name': 'electricity-theft-detection-using-machine-learning-IJERTCONV10IS04024.pdf',
  'media_link': 'https://api.docparser.com/v1/document/media/lin7He2PPDPdAoIHbLLl3i3efaZVYkEKiQyu5qVX1Ah3alakMyntbBQaYV03A5jRTBjON-ogp_eW5dqfJw8rvR5rh-dkdqzHAsD8KyldkEk',
  'media_link_original': 'https://api.docparser.com/v1/document/media/lin7He2PPDPdAoIHbLLl3i3efaZVYkEKiQyu5qVX1Ah3alakMyntbBQaYV03A5jRTBjON-ogp_eW5dqfJw8rvR5rh-dkdqzHAsD8KyldkEk/original',
  'media_link_data': 'https://api.docparser.com/v1/document/media/lin7He2PPDPdAoIHbLLl3i3efaZVYkEKiQyu5qVX1Ah3alakMyntbBQaYV03A5jRTBjON-ogp_eW5dqfJw8rvR5rh-dkdqzHAsD8KyldkEk/data',
  'page_count': 4,
  'uploaded_at': '2024-01-24T15:50:35+00:00',
  'processed_at': '2024-01-24T15:50:36+00:00',
  'uploaded_at_utc': '2024-01-24T15:50:35+00:00',
  'uploaded_at_user': '2024-01-24T07:50:35+00:00',
  'processed_at_utc': '2024-01-24T15:5

In [107]:
keywords = data[0]['keywords']
references = data[0]['references']
# abstract = data[0]['abstract']

In [108]:
def convert_to_file(text, file_path):
    
    with open(file_path, 'w') as file:
        file.write(text)

    return file_path

def convert_to_plaintext(input_string):
    # Use regular expression to remove non-alphabetic characters
    clean_string = re.sub(r'[^a-zA-Z ]', '', input_string)
    return clean_string

In [109]:
convert_to_file(references, 'extracted_text/abc.txt')

'extracted_text/abc.txt'

In [110]:
def extract_words_in_quotes(input_string):
    clean = re.sub(r'\n', ' ', input_string)
    pattern = r'[“"]([^”"]*)[”"]'
    matches = re.findall(pattern, clean)
    
    return matches

In [111]:
names = extract_words_in_quotes(references)
names[1]

'Electricity theft detection in AMI using customers consumption patterns,'

Scraping

In [112]:
def get_abstract_from_paper_url(paper_page_url, headers):
    response = requests.get(paper_page_url, headers=headers)

    if response.status_code == 200:
        paper_soup = BeautifulSoup(response.text, 'html.parser')

        abstract_element = paper_soup.find('div', {'class': 'nova-legacy-e-text nova-legacy-e-text--size-m nova-legacy-e-text--family-sans-serif nova-legacy-e-text--spacing-none nova-legacy-e-text--color-grey-800 research-detail-middle-section__abstract'})
        
        if abstract_element:
            abstract = abstract_element.get_text(strip=True)
            return abstract
        else:
            return "Abstract not found."
    else:
        return f"Failed to retrieve paper page: {response.status_code}"

In [123]:
def open_paper_link_by_name(paper_name):
    base_url = "https://www.researchgate.net/search/publication?q="
    search_url = base_url + paper_name.replace(" ", "+")
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    }


    # Make a request to the ResearchGate search page
    response = requests.get(search_url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract all search results
        result_links = soup.find_all('a', {'class':  ['nova-legacy-e-link', 'nova-legacy-e-link--color-inherit', 'nova-legacy-e-link--theme-bare']})

        if result_links:
            print("Resutls found")

            for link in result_links:
                result_name = link.get_text(strip=True)
                # print("search: ",result_name)

                result_name = convert_to_plaintext(result_name)
                # print("plain_name:" ,result_name)

                # Check if the result name matches the given paper name
                if paper_name.lower() in result_name.lower():
                    paper_page_url = "https://www.researchgate.net/" + link['href']
                    abstract = get_abstract_from_paper_url(paper_page_url, headers=headers)
                    print("full link: ",paper_page_url)
                    print("abstact: ",abstract)

                    return result_name, paper_page_url, abstract


            print(f"No exact match found for '{paper_name}' in the search results.")    
            # return None, None, None
        else:
            return f"No results found for '{paper_name}'."
    else:
        return f"Failed to perform the search: {response.status_code}"

In [114]:
names[3]

'Scikit-learn: Machine learning in Python,'

In [124]:
paper_name_to_search = convert_to_plaintext(names[3])
result = open_paper_link_by_name(paper_name_to_search)


Resutls found
No exact match found for 'Scikitlearn Machine learning in Python' in the search results.


In [116]:
print(result)

None


In [122]:
if result:
    name, link, abstract1 = result[:3]
    print()
else:
    print("No result found")


No result found


NameError: name 'abstract1' is not defined

In [1]:
mydict = {'1': {'Reference no': 1, 'Name': 'Energy Theft Detection With Energy Privacy Preservation in the Smart Grid', 'ResearchGate link': 'https://www.researchgate.net/publication/331578806_Energy_Theft_Detection_With_Energy_Privacy_Preservation_in_the_Smart_Grid?_sg=CHNbq5vRYNfl1NnbdKLsqWdK5FxfEtqexBy3ntS-kWJd0geM86FoSP3P_vSAXwL_eaI-i-vmo6vptBY', 'Abstract': 'As a prominent early instance of the IoT in the smart grid, the advanced metering infrastructure (AMI) provides real-time information from smart meters to both grid operators and customers, exploiting the full potential of demand response. However, the newly-collected information without security protection can be maliciously altered and result in huge loss. In this paper, we propose an energy theft detection scheme with energy privacy preservation in the smart grid. Specially, we use combined convolutional neural networks (CNN) to detect abnormal behavior of the metering data from a long-period pattern observation. In addition, we employ Paillier algorithm to protect the energy privacy. In other words, the users’ energy data are securely protected in the transmission and the data disclosure is minimized. Our security analysis demonstrates that in our scheme data privacy and authentication are both achieved. Experimental results illustrate that our modified CNN model can effectively detect abnormal behaviors at an accuracy up to 92.67%.'}, '2': {'Reference no': 2, 'Name': 'Electricity Theft Detection in AMI Using Customers Consumption Patterns', 'ResearchGate link': 'https://www.researchgate.net/publication/277897380_Electricity_Theft_Detection_in_AMI_Using_Customers%27_Consumption_Patterns?_sg=1OhuT7ByTQgsaECe9Ox9RWKjX06X13wu2FPGGddlzmLgW9vLchM_gJunoevCvR_mJDtvg4uLrBByLl8', 'Abstract': "As one of the key components of the smart grid, advanced metering infrastructure brings many potential advantages such as load management and demand response. However, computerizing the metering system also introduces numerous new vectors for energy theft. In this paper, we present a novel consumption pattern-based energy theft detector, which leverages the predictability property of customers' normal and malicious consumption patterns. Using distribution transformer meters, areas with a high probability of energy theft are short listed, and by monitoring abnormalities in consumption patterns, suspicious customers are identified. Application of appropriate classification and clustering techniques, as well as concurrent use of transformer meters and anomaly detectors, make the algorithm robust against nonmalicious changes in usage pattern, and provide a high and adjustable performance with a low-sampling rate. Therefore, the proposed method does not invade customers' privacy. Extensive experiments on a real dataset of 5000 customers show a high performance for the proposed method."}, '3': {'Reference no': 3, 'Name': 'PPETD PrivacyPreserving Electricity Theft Detection Scheme with Load Monitoring and Billing for AMI Networks', 'ResearchGate link': 'https://www.researchgate.net/publication/334061879_PPETD_Privacy-Preserving_Electricity_Theft_Detection_Scheme_with_Load_Monitoring_and_Billing_for_AMI_Networks?_sg=jhgcwQgmC_rX3foD7DJbt8ABRbuH3alNqeyqi5mEdScfdJJHezQ9_e2gJadlqNPBSVFu0HGV1DDH40Y', 'Abstract': 'In advanced metering infrastructure (AMI) networks, smart meters installed at the consumer side should report fine-grained power consumption readings (every few minutes) to the system operator for billing, real-time load monitoring, and energy management. On the other hand, AMI networks are vulnerable to cyber-attacks where malicious consumers report false (low) electricity consumption to reduce their bills in an illegal way. Therefore, it is imperative to develop schemes to accurately identify the consumers that steal electricity by reporting false electricity usage. Most of the existing schemes rely on machine learning for electricity theft detection using the consumers’ fine-grained power consumption meter readings. However, this fine-grained data that is used for electricity theft decetion, load monitoring, and billing can also be misused to infer sensitive information regarding the consumers such as whether they are on travel, the appliances they use, etc. In this paper, we propose an efficient and privacy-preserving electricity theft detection scheme for AMI network and we refer to it as PPETD. Our scheme allows system operators to identify the electricity thefts, monitor the loads, and compute electricity bills efficiently using masked fine-grained meter readings without violating the consumers’ privacy. PPETD uses secret sharing to allow the consumers to send masked readings to the system operator such that these readings can be aggregated for the purpose of monitoring and billing. In addition, secure two-party protocols using arithmetic and binary circuits are executed by the system operator and each consumer to evaluate a generalized convolutionalneural network model on the reported masked fine-grained power consumption readings for the purpose of electricity theft detection. An extensive analysis on real datasets is performed to evaluate the security and the performance of PPETD. Our results confirm that our scheme is accurate in detecting fraudulent consumers with privacy preservation and acceptable communication and computation overhead.'}, '5': {'Reference no': 5, 'Name': 'PrivacyPreserving Energy Theft Detection in Microgrids A State Estimation Approach', 'ResearchGate link': 'https://www.researchgate.net/publication/277618504_Privacy-Preserving_Energy_Theft_Detection_in_Microgrids_A_State_Estimation_Approach?_sg=ZUlTkK_5W4Sg8XpARTT-TJAORxmIqF9bUWhE1r_ciXiZrkQ8uXaSYna9xD7qCd983WkuKGRh_k5EMJ0', 'Abstract': "Energy theft is a notorious problem in electric power systems, which causes great economic losses and threatens the reliability of the power grid. Recently, the Smart Grid has been proposed as the next-generation power system to modernize the current grid and improve its efficiency, sustainability, and security. Key technologies of the Smart Grid include smart meters, which allow system operators to collect real-time power consumption data from users, and microgrids, which allow users to own and control renewable resources. However, the Smart Grid is vulnerable to cyber attacks, thus making stealing energy much easier in it. Most existing energy theft detection schemes require the collection of real-time power consumption data from users, i.e., users' load profiles, which violates their privacy. In this paper, we first propose a centralized energy theft detection algorithm utilizing the Kalman filter, called SEK. It can efficiently identify the energy thieves but cannot protect users' privacy. Then, based on SEK, we develop a privacy-preserving energy theft detection algorithm called PPBE, which privately finds the energy thieves by decomposing the Kalman filter into two parallel and loosely coupled filters. Finally, we conduct thorough privacy analysis and extensive simulations to validate our proposed algorithms."}, '6': {'Reference no': 6, 'Name': 'Energy Theft Detection With Energy Privacy Preservation in the Smart Grid', 'ResearchGate link': 'https://www.researchgate.net/publication/331578806_Energy_Theft_Detection_With_Energy_Privacy_Preservation_in_the_Smart_Grid?_sg=r99arwOSCdaATx6FD8HvJWO-eWOwGZ5tuI2ScjQGf5wAOdaZbGbiGX2ZzkYLb9UEDYYH9phqtzpg3Dw', 'Abstract': 'As a prominent early instance of the IoT in the smart grid, the advanced metering infrastructure (AMI) provides real-time information from smart meters to both grid operators and customers, exploiting the full potential of demand response. However, the newly-collected information without security protection can be maliciously altered and result in huge loss. In this paper, we propose an energy theft detection scheme with energy privacy preservation in the smart grid. Specially, we use combined convolutional neural networks (CNN) to detect abnormal behavior of the metering data from a long-period pattern observation. In addition, we employ Paillier algorithm to protect the energy privacy. In other words, the users’ energy data are securely protected in the transmission and the data disclosure is minimized. Our security analysis demonstrates that in our scheme data privacy and authentication are both achieved. Experimental results illustrate that our modified CNN model can effectively detect abnormal behaviors at an accuracy up to 92.67%.'}, '7': {'Reference no': 7, 'Name': 'Deep Recurrent Electricity Theft Detection in AMI Networks with Random Tuning of Hyperparameters', 'ResearchGate link': 'https://www.researchgate.net/publication/327495910_Deep_Recurrent_Electricity_Theft_Detection_in_AMI_Networks_with_Random_Tuning_of_Hyper-parameters?_sg=DtZOPFY18j2UYTE8x2oWnxFZrojaWDs1vBy0lGE6r_t54-SvqS_kF23gCQD0YSk49tvb0uy1374eQFY', 'Abstract': None}, '8': {'Reference no': 8, 'Name': 'PrivacyPreserving Energy Theft Detection in Smart Grids A PP Computing Approach', 'ResearchGate link': 'https://www.researchgate.net/publication/260580600_Privacy-Preserving_Energy_Theft_Detection_in_Smart_Grids_A_P2P_Computing_Approach?_sg=XbyyD7AyuR-bvYNyiGEr_R1vKagZ08L72nRCjd7ZDv_ldwPfXJQGvpdtLvHICD0jK4Dwf0cj8YZvpcA', 'Abstract': None}}

In [2]:
import csv
csv_filename = 'mydict_data.csv'

# Open the CSV file in write mode
with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    # Create a CSV writer object
    csv_writer = csv.writer(csvfile)

    # Write the header row
    csv_writer.writerow(['Reference no', 'Name', 'Link', 'Abstract'])

    # Iterate over each item in the dictionary
    for key, value in mydict.items():
        # Write a row for each dictionary entry
        csv_writer.writerow([value['Reference no'], value['Name'], value['ResearchGate link'], value['Abstract']])

print(f"Data saved to {csv_filename} successfully.")

{'1': {'Reference no': 1,
  'Name': 'Energy Theft Detection With Energy Privacy Preservation in the Smart Grid',
  'ResearchGate link': 'https://www.researchgate.net/publication/331578806_Energy_Theft_Detection_With_Energy_Privacy_Preservation_in_the_Smart_Grid?_sg=CHNbq5vRYNfl1NnbdKLsqWdK5FxfEtqexBy3ntS-kWJd0geM86FoSP3P_vSAXwL_eaI-i-vmo6vptBY',
  'Abstract': 'As a prominent early instance of the IoT in the smart grid, the advanced metering infrastructure (AMI) provides real-time information from smart meters to both grid operators and customers, exploiting the full potential of demand response. However, the newly-collected information without security protection can be maliciously altered and result in huge loss. In this paper, we propose an energy theft detection scheme with energy privacy preservation in the smart grid. Specially, we use combined convolutional neural networks (CNN) to detect abnormal behavior of the metering data from a long-period pattern observation. In addition, 